# Azure OpenAI SDK
This first example shows how to create a basic chatbot using the SDK.

## A. Basic BOT


1. Import SDK

In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.5"

2. Load Configuration

In [ ]:
// Import the helper class to load configuration values
#!import ../Config/TheHelper.cs

// Load configuration values
string endpoint = TheHelper.GetConfigurationValue("endpoint");
string apiKey = TheHelper.GetConfigurationValue("apikey");
string deploymentName = TheHelper.GetConfigurationValue("DeploymentName");

3. Create ApeinAI Client

In [ ]:
// Import the necessary namespaces
using Azure;
using Azure.AI.OpenAI;
using static System.Environment;
using Microsoft.DotNet.Interactive;

// Create a new OpenAI client using the specified endpoint and API key
OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(apiKey));

// Print the endpoint and engine name to the console
Console.Write($"endpoint: {endpoint}\n");
Console.Write($"engine: {deploymentName}\n");

4. Set the Assistan using System message (Prompt)

The `system message` is used to provide contextual instructions or guidelines to the language model before generating a response. It helps steer the `model's behavior` and can be used to specify the desired format, style, or tone of the response. The system message is typically included at the beginning of the conversation and serves as an input prompt for the model.


In [ ]:
// Define the chat completion options
var chatCompletionsOptions = new ChatCompletionsOptions()
{
    Messages =
    {
        // Send a system message to set the context
        new ChatMessage(ChatRole.System, "You are Rick from the TV show Rick & Morty. Pretend to be Rick."),
        
        // Send a user message to prompt for an introduction
        new ChatMessage(ChatRole.User, "Introduce yourself."),
    },
    MaxTokens = 100
};

// Send the chat completion options to the OpenAI API and get a response
Response<ChatCompletions> response = client.GetChatCompletions(deploymentOrModelName: deploymentName, chatCompletionsOptions);

// Print the response to the console
Console.WriteLine("Rick: " + response.Value.Choices[0].Message.Content);
Console.WriteLine();

# B. Stream the chat response 


This code prompts the user for input and sends it to the OpenAI API as a chat message. It then receives and prints chat responses in an infinite loop. The chat messages are stored in a `ChatCompletionsOptions` object, which is sent to the OpenAI API using a `GetChatCompletionsStreamingAsync` method. The chat responses are received using a `GetChoicesStreaming` method and printed to the console using a `StringBuilder` object. The chat responses are then added to the `ChatCompletionsOptions` object as an assistant message, and the loop continues.

In [ ]:
// Prompt the user for input and send it to the OpenAI API as a chat message
Console.Write("Morty prompt: ");
var userMessage = await Kernel.GetInputAsync("Morty prompt:");
Console.WriteLine(userMessage);

chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, userMessage));

// Start an infinite loop to receive chat responses from the OpenAI API
while (true)
{
    Console.Write("Rick: ");

    // Send the chat completion options to the OpenAI API and get a streaming response
    var chatCompletionsResponse = await client.GetChatCompletionsStreamingAsync(deploymentName, chatCompletionsOptions);

    // Build a string containing the chat response and print it to the console
    var chatResponseBuilder = new StringBuilder();
    await foreach (var chatChoice in chatCompletionsResponse.Value.GetChoicesStreaming())
    {
        await foreach (var chatMessage in chatChoice.GetMessageStreaming())
        {
            chatResponseBuilder.Append(chatMessage.Content);
            Console.Write(chatMessage.Content);
            await Task.Delay(TimeSpan.FromMilliseconds(200));
        }
    }

    // Add the chat response to the chat completion options as an assistant message
    chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.Assistant, chatResponseBuilder.ToString()));

    Console.WriteLine();

    // Prompt the user for input and send it to the OpenAI API as a chat message
    Console.Write("Morty prompt: ");
    var userMessage = await Kernel.GetInputAsync("Morty prompt:");
    Console.Write(userMessage);

    chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, userMessage));
    Console.WriteLine();
}

Next Sample [Prompt with clear instructions](./2.ClearInstructions.ipynb)